In [1]:
import os
import json
import numpy as np
import pandas as pd
import random
import gymnasium as gym
import matplotlib.pyplot as plt
from matplotlib import animation
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from IPython import display

In [6]:
data_directory = 'states'

X, y = [], []
for filename in os.listdir(data_directory):
  
  if not filename.startswith('expert') or not filename.endswith('.txt'):
    continue

  print(2)
  with open(f'{data_directory}/{filename}', 'r') as file:
    content = json.load(file)
    memory, action = zip(*content)
    X.extend(memory)
    y.extend(action)

X, y = pd.DataFrame(X), pd.DataFrame(y)
X

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
1,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
2,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
3,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
4,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54955,0,114,81,80,83,9,82,81,26,89,...,32,58,96,9,0,0,98,250,110,215
54956,0,114,81,80,83,9,82,81,26,89,...,32,58,96,9,0,0,98,250,110,215
54957,0,114,81,80,83,9,82,81,26,89,...,32,58,96,9,0,0,98,250,110,215
54958,0,114,81,80,83,9,82,81,26,89,...,32,58,96,9,0,0,98,250,110,215


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)

model = CatBoostClassifier(iterations=1500, depth=2, verbose=True, loss_function='MultiClass', learning_rate=4e-2)
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 1.5568660	test: 1.5560894	best: 1.5560894 (0)	total: 164ms	remaining: 9m 35s
1:	learn: 1.5104608	test: 1.5090398	best: 1.5090398 (1)	total: 436ms	remaining: 12m 42s
2:	learn: 1.4700138	test: 1.4680992	best: 1.4680992 (2)	total: 629ms	remaining: 12m 12s
3:	learn: 1.4343074	test: 1.4320335	best: 1.4320335 (3)	total: 807ms	remaining: 11m 45s
4:	learn: 1.4022787	test: 1.3997339	best: 1.3997339 (4)	total: 932ms	remaining: 10m 51s
5:	learn: 1.3736974	test: 1.3709057	best: 1.3709057 (5)	total: 1.06s	remaining: 10m 16s
6:	learn: 1.3482026	test: 1.3450528	best: 1.3450528 (6)	total: 1.19s	remaining: 9m 52s
7:	learn: 1.3251835	test: 1.3217086	best: 1.3217086 (7)	total: 1.29s	remaining: 9m 25s
8:	learn: 1.3043782	test: 1.3006005	best: 1.3006005 (8)	total: 1.41s	remaining: 9m 5s
9:	learn: 1.2850556	test: 1.2811159	best: 1.2811159 (9)	total: 1.52s	remaining: 8m 51s
10:	learn: 1.2678191	test: 1.2636135	best: 1.2636135 (10)	total: 1.68s	remaining: 8m 53s
11:	learn: 1.2527900	test: 1.2482271	

In [8]:
env = gym.make('ALE/MsPacman-ram-v5', render_mode='rgb_array')
observation, info = env.reset()
observation, reward, terminated, truncated, info = env.step(0)

observations = []

while not terminated and not truncated:
  action = model.predict(observation)[0]
  observation, reward, terminated, truncated, info = env.step(action)
  observations.append(env.render())

c:\Users\user\miniconda3\envs\myenv2\Lib\site-packages\gymnasium\utils\passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Визуализация предсказаний

In [9]:
video = np.array(observations)

figure = plt.figure()
images = plt.imshow(video[0, :, :, :])
plt.close()

def init():
  images.set_data(video[0, :, :, :])

def animate(i):
  images.set_data(video[i, :, :, :])
  return images

html_video = animation.FuncAnimation(figure, animate, init_func=init, frames=video.shape[0], interval=50).to_html5_video()
display.HTML(html_video)